In [1]:
!nvidia-smi

Sun May  2 01:14:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Preparations

requirements

In [ ]:
!pip install transformers==4.4.0
!pip install pydub
!apt install ffmpeg
!pip install datasets
!pip install jiwer
!pip install python-Levenshtein

covert raw csv to compatible csv format

In [12]:
# covert to compatible csv file

df = pandas.read_table("knnw_en_sub_edit.csv", sep = ";", header=0)
df.to_csv("knnw_sub.csv", index=False)

train/val split

In [295]:
df = pd.read_table("knnw_en_sub_edit.csv", sep = ";", header=0)
train_df = df.sample(frac=0.8)
train_df.to_csv("knnw_sub_train.csv", index=False)

val_df = df.drop(train_df.index)
val_df.to_csv("knnw_sub_val.csv", index=False)

prepare wav files

In [ ]:
audio_path = "/path/to/knnw_en_mono.wav"
subtitle_lookup_path = "knnw_sub.csv"
save_dir = "./wav_data/"

In [83]:
def preprocess_wav(audio_path, subtitle_lookup_path, save_dir):
    audio = AudioSegment.from_file(audio_path, format="wav")
    total_duration = len(audio)
    subtitle_lookup = pd.read_table(subtitle_lookup_path, sep = ";", header=0)  # drop the first non-dialogue subtitle
    for i in range(len(subtitle_lookup)):
        start_time = subtitle_lookup.iloc[i, 1]
        stop_time = subtitle_lookup.iloc[i, 2]

        audio_item = audio[start_time: stop_time]
        audio_item.export(save_dir + str(subtitle_lookup.iloc[i, 0]) + ".wav", format="wav")

preprocess_wav("./", subtitle_lookup_path)

# Wav2Vec 2.0


In [ ]:
import re
import json
import sys
import time
import random

import numpy as np
import pandas as pd

import torch

from pydub import AudioSegment
import librosa

import Levenshtein

from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC

from datasets import load_dataset, load_metric
from datasets import ClassLabel
from IPython.display import display, HTML

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## preprocess data

load train/test data

In [ ]:
knnw = load_dataset('csv', data_files={'train': './knnw_sub_train.csv', 'test': './knnw_sub_val.csv'})

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(knnw["train"])

data cleaning

In [306]:
def remove_chars(batch):
    batch['Text'] = batch['Text'].lower()
    null = 'null'
    batch['Text'] = re.sub(r'.*""', null, batch['Text'])
    batch['Text'] = batch['Text'].replace('?', '')
    batch['Text'] = batch['Text'].replace('!', '')
    batch['Text'] = batch['Text'].replace(',', '')
    batch['Text'] = batch['Text'].replace('-', ' ')
    batch['Text'] = batch['Text'].replace('"', '')
    batch['Text'] = batch['Text'].replace("“", '')
    batch['Text'] = batch['Text'].replace("”", '')
    batch['Text'] = batch['Text'].replace('...', '')
    batch['Text'] = batch['Text'].replace('é', 'e')
    batch['Text'] = batch['Text'].replace('21', 'twenty one')
    batch['Text'] = batch['Text'].replace('1200', 'twelve hundred')
    batch['Text'] = batch['Text'].replace('20th', 'twentieth')
    batch['Text'] = batch['Text'].replace('7:40', 'seven fourty')
    batch['Text'] = batch['Text'].replace('8:42', 'eight fourty two')
    batch['Text'] = batch['Text'].replace('1994', 'nineteen ninety four')
    batch['Text'] = batch['Text'].replace('9', 'nine')
    batch['Text'] = batch['Text'].replace('500', 'five hundred')
    batch['Text'] = re.sub(r'\(.*\)', '', batch['Text'])
    batch['Text'] = re.sub(r'[\w ]+: ', ' ', batch['Text'])
    batch['Text'] = re.sub(r' +', ' ', batch['Text'])
    if batch['Text'][0] == ' ':
        batch['Text'] = batch['Text'][1:]
    batch['Text'] = re.sub(r'\[.*\] *', ' ', batch['Text'])
    if batch['Text'] == '':
        batch['Text'] = null
    
    # changes from our hw4p2 remove_chars():
    batch['Text'] = '<s>' + batch['Text'].upper() + '</s>'
    batch['Text'] = batch['Text'].replace(" ", "|")
    batch['Text'] = batch['Text'].replace(".", "")
    
    return batch


In [ ]:
knnw = knnw.map(remove_chars)


In [ ]:
show_random_elements(knnw['train'])


In [270]:
## Using pretrained processor instead. No need to construct our own vocab

# def extract_all_chars(batch):
#   all_text = " ".join(batch["Text"])
#   vocab = list(set(all_text))
#   return {"vocab": [vocab], "all_text": [all_text]}

# vocabs = knnw.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=knnw.column_names["train"])

# vocab_list = list(set(vocabs["train"]["vocab"][0]))

# vocab_dict = {v: k for k, v in enumerate(vocab_list)}
# # vocab_dict["|"] = vocab_dict[" "]
# # del vocab_dict[" "]
# vocab_dict["<unk>"] = len(vocab_dict)
# vocab_dict["<pad>"] = len(vocab_dict)
# vocab_dict["<s>"] = len(vocab_dict)
# vocab_dict["</s>"] = len(vocab_dict)
# vocab_dict

# import json
# with open('vocab.json', 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)

load wav files into arrays

In [ ]:
import pydub
wav_dir = './wav_data/'

def speech_file_to_array_fn(batch):
    speech_array, sr = librosa.load(wav_dir + str(batch['Number']) + ".wav", sr=16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sr
    batch["target_text"] = batch["Text"]
    return batch

knnw = knnw.map(speech_file_to_array_fn, remove_columns=knnw.column_names["train"], num_proc=4)

check examples

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(knnw["train"]))
print(knnw["train"][rand_int]["target_text"])
ipd.Audio(data=np.asarray(knnw["train"][rand_int]["speech"]), autoplay=True, rate=16000)


In [312]:
rand_int = random.randint(0, len(knnw["train"]))

print("Target text:", knnw["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(knnw["train"][rand_int]["speech"]).shape)
print("Sampling rate:", knnw["train"][rand_int]["sampling_rate"])

Target text: <s>MUST|BE|NICE</s>
Input array shape: (22720,)
Sampling rate: 16000


In [313]:
# tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token='<unk>', pad_token="<pad>", word_delimiter_token=" ")
# feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")


convert chars into ids

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

knnw_prepared = knnw.map(prepare_dataset, remove_columns=knnw.column_names["train"], batch_size=8, num_proc=4, batched=True)


## setup trainer

In [315]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [316]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [317]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    pred_str = [s.replace("<s>", "").replace("</s>", "") for s in pred_str]
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    label_str = [s.replace("<s>", "").replace("</s>", "") for s in label_str]

    distances = np.vectorize(lambda x,y: Levenshtein.distance(x,y))(pred_str, label_str)
    print(pred_str)
    print(label_str)

    distance = np.mean(distances)

    return {"edit_distance": distance}

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h", #"facebook/wav2vec2-base", 
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)


In [319]:
model.freeze_feature_extractor()

In [320]:
model = model.to(device)

In [321]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./wav2vec2-base-knnw-demo",
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=1e-4,
  weight_decay=0.000005,
  warmup_steps=10,
  save_total_limit=2,
)

In [322]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=knnw_prepared["train"],
    eval_dataset=knnw_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [323]:
trainer.train()

Step,Training Loss,Validation Loss,Edit Distance,Runtime,Samples Per Second
200,1.646600,1.415815,7.910506,15.617600,16.456000
400,0.883800,1.206095,7.042802,15.615800,16.458000
600,0.631000,1.234995,6.680934,15.665800,16.405000
800,0.493100,1.136785,6.634241,15.689600,16.380000
1000,0.400400,1.220966,6.622568,15.899200,16.164000
1200,0.382500,1.232310,6.474708,15.987000,16.076000
1400,0.306800,1.215112,6.214008,15.499700,16.581000
1600,0.285300,1.283225,6.225681,15.915400,16.148000


['IT ONLY AR VOICE IS TR TO NIGHT', "WHAT'SI ITS O LETHER ON THE COWN A THRE", "HI'M TOLE THAT SOME PART OF EVERY WISH WILL BE HEARD", 'IM A WAWAY TO YI', 'TUSIN AUTO YOUR NAME', 'TAK', "YOU WON'TTA HAVE ANYRS LIHES FISH GRANDMA", "RAZY BOUGHT YOU'R SO LOW", 'THE MOR', 'LYING', "LET HOR MENING HAS YOU'RE HAVY", 'NO INSO', 'EK OURSILSE SOMETHING TOTALLY POSSESSED YOUR BODY', 'WHOALD DOUD JUST GIVE IT A REST WITH ALL THE SECULT NANSES', 'AS THE ENCOMBENT MY ADMINISTRATION NAT OF EASINESS GETTIN GET ANOTHER CURM ANNYWY', 'O HE STUFF ON HIS FAMIL', 'WELL ITH THE GOLDEN OUR AND MAGIC HOUR ARE TECHTICALLY THE SAME', 'NEX', 'WHAT NO WAY', "IK COULD DREAM ABOUT THAT SOMEONE ELSE'S LIFE", "IT'S ALL O FORRY", 'I WANT A GRADUATE ALREADY AND PUT A TAK TO A SORT AS IDE TIM', 'NO JOBS IN TOWN O ONE DEEDU', 'EVEN THE DAYS ARE TOO SURE', 'OU KNOW WHAT', 'NOW OW WHAT', 'WHAT ARE YOU GOIN TO DO LEFT TOR OA GRADUATE HIGH SCHOOL', 'THE WANTED DO AT YOUR DOIN', "'S NOT LIKE THREAD'S TAK SHE'S TALLING ITA F

TrainOutput(global_step=1650, training_loss=0.6175197358564897, metrics={'train_runtime': 2738.972, 'train_samples_per_second': 0.602, 'total_flos': 1.5587488819961856e+18, 'epoch': 50.0, 'init_mem_cpu_alloc_delta': 947735113, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 325647997, 'init_mem_gpu_peaked_delta': 1522651648, 'train_mem_cpu_alloc_delta': 580047, 'train_mem_gpu_alloc_delta': 1114872320, 'train_mem_cpu_peaked_delta': 134030961, 'train_mem_gpu_peaked_delta': 7121912832})

In [324]:
trainer.train()

Step,Training Loss,Validation Loss,Edit Distance,Runtime,Samples Per Second
200,0.275200,1.281971,6.217899,16.554900,15.524000
400,0.301200,1.281971,6.217899,15.751900,16.315000
600,0.290200,1.281971,6.217899,15.899200,16.164000
800,0.301400,1.281971,6.217899,15.446900,16.638000
1000,0.292100,1.281971,6.217899,15.653800,16.418000
1200,0.276600,1.281971,6.217899,16.959200,15.154000
1400,0.286300,1.281971,6.217899,15.830800,16.234000
1600,0.291600,1.281971,6.217899,16.991100,15.126000


['I FONLY OUR VOICES DART THENITE', 'LOOT YT TOGETHER AND COUNTEC FOR Y', "I'M TOLD THAT SOMEBONY EVERY WISH WILL BE HURD", "I'M ON MY WAY TO YEUH", 'DI SINGA TO YOR NAME', 'TAKI', "YOU WANNA HAVE ANY LAST NIGHT'S FISH GRANDMA", "LAZY BUT YOU'RE SO SLO", "GOO MORNIN'", "WE'RE LETHING", "WHAT MORE MENING HAUSE YOU'RE HEAVY", "NO IT DON'", 'TIK ORE SIENCED SOMETHING TOTALLY POSSESSED YOUR BOD', 'WHULD DHOUD JUST GIVE IT A REST WITH ALL THE SAKALT MANTES', 'AS THE ENCOMBENT MY ADMINISTRATION MAG OF EASINESS GENNI GET ANOTHER TERM ANYWAY', 'OH HE STUFF ON HIS FAMILY', 'WELL THE GOLDEN HOUR AND MAGIC HOUR ARE TECHTICALLY THE SAME', 'OH HEY NAG', 'WHAT NO WAY', "I COULD DREAM ABOUT SOMEONE ELSE'S LIFE", "IT'S ALL O FUNLY", 'I WANT A GRADUATE ALREADY AND GO TO TOKU A SORT AS I JAM', 'NO JOBS IN TOWN OR ONE DETO', 'EVEN THE DAYS ARE TOO SORT', 'YO KNOW WHAT', 'NO OH WHAT', 'WHAT ARE YOU GONNA DO LEFT IN OUR GRADUATE HIGH SCHOOL', "I WANNA DO WHAT YOU ARE DOIN'", "'S NOT LIKE THREAD'ST TALK SHE

TrainOutput(global_step=1650, training_loss=0.2893476220333215, metrics={'train_runtime': 2801.6157, 'train_samples_per_second': 0.589, 'total_flos': 1.5575639287670784e+18, 'epoch': 50.0, 'train_mem_cpu_alloc_delta': 585416, 'train_mem_gpu_alloc_delta': 1281024, 'train_mem_cpu_peaked_delta': 132153910, 'train_mem_gpu_peaked_delta': 7148775424})

# Evaluate

In [ ]:
def map_to_result(batch):
  model.to("cuda")
  input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt"
  ).input_values.to(device)

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]#.lower()

  return batch

results = knnw["train"].map(map_to_result)

In [166]:
# compute Levenshtein distance

distances = np.vectorize(lambda x,y: Levenshtein.distance(x,y))(results['target_text'], results["pred_str"])
avg_distance = sum(distances) / len(results['target_text'])

avg_distance

In [ ]:
results['pred_str'][:5]

In [ ]:
results['target_text'][:5]